In [21]:
import pandas as pd
import nltk
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer#, tfidfconverter
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import regex as re
import requests
from sklearn.svm import SVC

In [22]:
# Load data into a dataframe
addr = "/Users/tthekkum/Documents/LnD/BV/550_DataMining/Second time/week9_10/categorized-comments.jsonl"
data1 = pd.read_json(addr ,lines = True)

In [23]:
data = data1.head(10000)

In [24]:
data=data1

In [25]:
data.cat.unique()

array(['sports', 'science_and_technology', 'video_games'], dtype=object)

In [26]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/tthekkum/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tthekkum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tthekkum/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tthekkum/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
from sklearn.model_selection import train_test_split, GridSearchCV
X = data['txt']
y = data['cat']

In [28]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    documents.append(document)

In [29]:
X1 = documents


In [30]:
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = tfidfconverter.fit_transform(X1).todense()

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 13, stratify=y)

In [32]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((485180, 1500), (485180,), (121295, 1500), (121295,))

In [33]:
from keras.layers import Dense
from keras.models import Sequential

N_FEATURES = 1500
N_CLASSES = 3

def build_network():
    """
    Create a function that returns a compiled neural network
    """
    nn = Sequential()
    nn.add(Dense(50, activation='relu'
                 , input_shape=(N_FEATURES,)
                 ))
    nn.add(Dense(15, activation='relu'))
    nn.add(Dense(N_CLASSES, activation='softmax'))
    nn.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return nn

In [34]:
from sklearn.pipeline import Pipeline
from transformer import TextNormalizer
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
pipe_mnnb = Pipeline([
        #('norm', TextNormalizer()),
        #('vect', TfidfVectorizer(max_features=N_FEATURES)).todense(),
        ('nn', KerasClassifier(build_fn=build_network,
                               epochs=2,
                               batch_size=10))
    ])


In [35]:
pipe_mnnb.fit(X_train, y_train)

Epoch 1/2
48518/48518 [==============================] - 38s 782us/step - loss: 0.4723 - accuracy: 0.7996

Pipeline(steps=[('nn',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fae5370cf98>)])

In [36]:
py_pred1 = pipe_mnnb.predict(X_test)

In [37]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,py_pred1))

[[ 2018   140  2864]
 [   63 13882 15220]
 [  537  4050 82521]]


In [38]:
print(classification_report(y_test,py_pred1))

                        precision    recall  f1-score   support

science_and_technology       0.77      0.40      0.53      5022
                sports       0.77      0.48      0.59     29165
           video_games       0.82      0.95      0.88     87108

              accuracy                           0.81    121295
             macro avg       0.79      0.61      0.67    121295
          weighted avg       0.81      0.81      0.79    121295



In [39]:
print(accuracy_score(y_test, py_pred1))



0.811418442639845
